In [ ]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('max_columns', None)

# Load data using `read_csv`

- 미국시장 재무제표 데이터 크롤링: https://nbviewer.jupyter.org/gist/FinanceData/35a1b0d5248bc9b09513e53be437ac42

In [ ]:
df = pd.read_csv("my_data/naver_finance/2015_12.csv")

In [ ]:
df.head()

# Exploratory Data Analysis (EDA)

- In statistics, exploratory data analysis (EDA) is an approach to analyzing data sets to summarize their main characteristics, often with visual methods(wiki)
- Two parts
    - *Metadata* : data about data. 
        - 데이터 크기
        - 컬럼명
        - 데이터 타입
        - 비어 있는 데이터
        - etc
    - *Univariate descriptive statistics*: summary statistics about individual variables(columns)


## Metadata

In [ ]:
df.shape

In [ ]:
df.get_dtype_counts()

In [ ]:
df.info()

In [ ]:
df['ticker'].dtype

- Rename columns

In [ ]:
df = df.rename(columns={"ticker": "종목명"})

In [ ]:
df.head()

## `describe()`

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
# Trnaspose (index <-> columns 뒤집기)
# 함수의 cascading을 통해서 한번에 진행 가능
df.describe().T

In [ ]:
# 1. numeric
df.describe(include=[np.number]).T  #  = df.describe()의 기본(default) 작동방식과 같습니다

In [ ]:
# dtype을 나타낼 때, string으로 해도 되고, library의 datatype으로 설정해도 됩니다. (astype() function을 쓸 때도 마찬가지)
# (아래 4개는 다 같은 구문 )
# df.describe(include=['int', 'float']).T 
# df.describe(include=['int64', 'float64']).T
# df.describe(include=[np.int64, np.float64]).T
# df.describe(include=['number']).T
# df.describe(include=[np.number]).T

In [ ]:
df.describe(percentiles=[0.01, 0.03, 0.99]).T.head(2)

In [ ]:
# 2. non-numeric (e.g. string, categorical)
df.describe(include=[np.object, pd.Categorical]).T.head()  # 'top'은 "가장 많이 출현하는 단어"를 의미함

- exclude

In [ ]:
df.describe(exclude=[np.number]).T.head()

- 참고: `quantile()` method

In [ ]:
df['PER(배)'].quantile(.2)
df['PER(배)'].quantile([.1, .2, .3])

## `unique(), value_counts()`

In [ ]:
# For DataFrame => nunique()
df.nunique()

In [ ]:
# For Series => unique(), nunique(), value_counts()
df['종목명'].unique()

In [ ]:
df['종목명'].nunique()

In [ ]:
df['종목명'].value_counts()

In [ ]:
df['종목명'].value_counts(normalize=True)

In [ ]:
# value_counts() ignore np.nan
a = pd.DataFrame({'a':[np.nan, 1, 2]})['a']
a

# 위의 코드는  아래와 같음
# a = pd.Series([np.nan, 1, 2])

In [ ]:
a.value_counts()

## example

In [ ]:
a = pd.read_csv("my_data/symbol_sector.csv", index_col=0)

In [ ]:
a.head()

In [ ]:
a.shape

In [ ]:
a['Sector'].nunique()

In [ ]:
a['Sector'].value_counts()

# 정렬


In [ ]:
df.head()

## top n
 

In [ ]:
df.nsmallest(5, "PER(배)")

In [ ]:
# PER이 가장작은 100개중에서, 그 중에서 당기순이익이 가장 큰 5개 종목의 데이터 
df.nsmallest(100, "PER(배)").nlargest(5, '당기순이익(억원)')

##  Sort
 

In [ ]:
df.sort_values("EPS(원)")

In [ ]:
df.sort_values("EPS(원)", ascending=False).head()

In [ ]:
df.sort_values(
    ['순이익률(%)', 'EPS(원)'],
    ascending=[True, False]
).head()

# Subset 추출하기

In [ ]:
df.head()

In [ ]:
df.shape

## By Columns 

In [ ]:
# string으로 인덱싱을 하면 -> Series로 반환한다
series = df['EPS(원)']

In [ ]:
# list로 인덱싱을 하면 -> DataFrame로 반환한다 
df2 = df[['EPS(원)', '종목명']]

In [ ]:
type(df['순이익률(%)'])                    # column명을 string으로 전달하면 -> Series 반환
type(df[['순이익률(%)', '당기순이익(억원)'] ]) # column명을 리스트로 전달하면 -> DAtaFrmae 반환  

- `filter()`

In [ ]:
df[['ROE(%)', 'ROA(%)', 'ROIC(%)']].head()

In [ ]:
df.filter(like="RO").head()

In [ ]:
df.filter(like="%").head()

In [ ]:
df.filter(like="P").head()

In [ ]:
df.filter(regex="P\w+R").head()

## By dtype

In [ ]:
df.get_dtype_counts()

In [ ]:
df.select_dtypes(include=['float']).head()

In [ ]:
df.select_dtypes(include=['object']).head()
# df.select_dtypes(include=['str']).head() (X) => use `object` instaed

## By Row

In [ ]:
name_df = df.set_index("종목명")
name_df.head()

### iloc, loc

In [ ]:
name_df.iloc[0]

In [ ]:
name_df.iloc[[0, 3]]

In [ ]:
name_df.loc["BYC"]

In [ ]:
name_df.loc[['삼성전자', 'CJ']]

### Select rows by prefix

In [ ]:
name_df.head()

In [ ]:
# 반드시 index를 sort를 해야만 loc을 이용한 range indexing이 가능
# index가 sort된 새로운 dataframe을 return하는데 그것을 다시 name_df로 받음
name_df = name_df.sort_index()

In [ ]:
name_df.index.is_monotonic_increasing

In [ ]:
name_df.loc["삼성":"삼성전자"].head(2)

In [ ]:
name_df.loc["가":"다"].head()

### More about and loc, iloc

In [ ]:
name_df.loc["삼성전자"]

In [ ]:
name_df['순이익률(%)']

In [ ]:
# 위의 둘을 동시에 하는 방법
name_df.loc["삼성전자", "순이익률(%)"]

# 권장하지 않는 방법
# name_df.loc["삼성전자"]["순이익률(%)"]

In [ ]:
name_df.loc[["삼성SDI", "삼성전자"], "순이익률(%)"]

In [ ]:
name_df.loc[["삼성SDI", "삼성전자"], ["순이익률(%)", "EPS(원)"]]

In [ ]:
# (index가 정렬이 된 경우만) -> range indexing(:)
name_df.loc["삼성":"삼성전자"]
name_df.loc["삼성":"삼성전자", :]
name_df.loc["삼성":"삼성전자", "순이익률(%)"]
name_df.loc["삼성":"삼성전자", ["순이익률(%)", "EPS(원)"]]

In [ ]:
name_df.iloc[[0, 3], :]
name_df.iloc[[0, 3], [0,1]]

# df.iloc[[0, 3], "상장일"]   # error
# df.iloc[[0, 3], ["상장일", "종가"]]   # error

### iloc, loc's return type

In [ ]:
# column indexing의 경우1. Series로 return
name_df['순이익률(%)'].head()

In [ ]:
# column indexing의 경우2. DataFrame으로 return
name_df[['순이익률(%)', 'EPS(원)']].head()

- For Series data

In [ ]:
a = pd.Series([1,2,3], index=['a', 'b', 'c'])
a

In [ ]:
a.iloc[0]     

In [ ]:
a.loc['a']  # = a['a']와 결과가 같음

In [ ]:
a.iloc[2]   # scalar
a.iloc[[2]]   # series

- For DataFrame Data -> column indexing에서와 동일한 원리

In [ ]:
df.iloc[2]   # Series
df.iloc[[2]]   # DataFrame

### For Scalar Value

- use `.at` or `.iat`

In [ ]:
df.loc[100, '순이익률(%)']

In [ ]:
df.at[100, '순이익률(%)']  

In [ ]:
## Much faster if use `.iat` or `.at`
# => Table이 크면 클수록 더 차이가 많이 남
%timeit df.loc[100, '순이익률(%)']
%timeit df.at[100, '순이익률(%)'] 

In [ ]:
# Also works with Series
%timeit df['순이익률(%)'].iloc[100]
%timeit df['순이익률(%)'].iat[100]

## Boolean selection
 

### Boolean Series

In [ ]:
tmp_series = pd.Series({"a":1, "b":2})
tmp_series

In [ ]:
tmp_series > 2

### Boolean DataFrame

In [ ]:
tmp_df = pd.DataFrame({
    'a':[1,np.nan,3,4,np.nan],
    'b':[5, 3, 3, 4,np.nan]
})
tmp_df

In [ ]:
tmp_df > 2

### DataFrame내에서 "Boolean Series" 만들기

In [ ]:
df.head()

In [ ]:
# 이번에는 Series와 Series를 비교
# 그 결과 새로운 boolean series이 생기게 되어 변수에 저장할 수 있게 됩니다.
a = df['순이익률(%)'] > df['영업이익률(%)']
a.head()

In [ ]:
a.sum()

In [ ]:
a.mean()

- Boolean series로 indexing 하기

In [ ]:
# row 갯수와, index 값이 서로 같음
df.shape
a.shape

- 대괄호를 이용하는 방법

In [ ]:
df[a].head()
df[a].shape

- loc를 이용하는 방법

In [ ]:
df.loc[a]    # df[a]와 결과가 같음
df.loc[a].shape

In [ ]:
df.loc[[0, 3]]

In [ ]:
# df.loc[a]
# => df.loc[[9, 10, 13 ....]] 이렇게 내부적으로 변환이 된다고 생각하시면 됩니다. True에 해당하는 row의 index만 가져와서 indexing을 하는 것이죠

In [ ]:
# shape의 변화
df.shape
df[df['순이익률(%)'] > df['영업이익률(%)']].shape
df.loc[df['순이익률(%)'] > df['영업이익률(%)']].shape

### Multiple boolean series

In [ ]:
con1 = df['순이익률(%)'] > df['영업이익률(%)']
con2 = df['PBR(배)'] < 1

In [ ]:
con1.head()
con2.head()

In [ ]:
True and True

if a > 1 and(or) b < 2:
    print("!")

- and = &
- or = |

In [ ]:
# [잘못된 방법] final_con = con1 and con2
# Pandas에서는 아래와 같이 조건식을 구성해야합니다.
final_con = con1 & con2

In [ ]:
final_con.head()

In [ ]:
df[final_con].head()
df[final_con].shape

In [ ]:
df[final_con].head(2)
df.loc[final_con].head(2)
df.loc[final_con, ['ROE(%)']].head(2)

# [X] df.iloc[final_con].head(2)

### `isin()`

In [ ]:
name_list = ['삼성전자', '현대건설', "삼성물산"]

1. multiple boolean series를 이용하는 방법

In [ ]:
cond1 = df['종목명'] == "삼성전자"
cond2 = df['종목명'] == "현대건설"
cond3 = df['종목명'] == "삼성물산"

final_con = cond1 | cond2 | cond3
df[final_con]

2. index화 해서 loc으로 row-wise indexing을 이용해서 가져오는 방법

In [ ]:
tmp_df = df.set_index('종목명')
tmp_df.head()

In [ ]:
tmp_df.loc[['삼성전자', '현대건설', "삼성물산"]]

3. isin() 함수를 이용해서 가져오는 방법

In [ ]:
cond = df['종목명'].isin(name_list)
df[cond]

In [ ]:
df[df['종목명'].isin(name_list)].head(2)
df.loc[df['종목명'].isin(name_list)].head(2)
df.loc[df['종목명'].isin(name_list), ['종목명', 'ROA(%)', 'ROE(%)']].head(2)

### `all()` vs `any()`

In [ ]:
a = df['순이익률(%)'] > 0

In [ ]:
a.all()

In [ ]:
a.any()

In [ ]:
(df['순이익률(%)'] > 0).all()

In [ ]:
(df['순이익률(%)'] > 0).any()

In [ ]:
# 왜 결과가 False일지 생각해보세요
(df['순이익률(%)'] > -1000000).all()

### example

In [ ]:
a = pd.read_csv("my_data/symbol_sector.csv", index_col=0)
a.head()

In [ ]:
a['Sector'].value_counts()

In [ ]:
a['Sector'].value_counts().nlargest(5)

In [ ]:
top_5_sector_list = a['Sector'].value_counts().nlargest(5).index
top_5_sector_list

In [ ]:
a[a['Sector'].isin(top_5_sector_list)] 

# 연산(Arithmetic)

In [ ]:
import FinanceDataReader as fdr
price_df = fdr.DataReader("005930", '2009-09-16', '2018-03-21')
price_df.head()

## 연산 기준

- DataFrame은 기준이 columns
- Series는 기준이 index
- 따로 명시가 없다면 Series의 index가 DataFrame의 columns에 맞춰짐!

## DataFrame & Series

In [ ]:
price_df.iloc[0]

In [ ]:
# Subtract row Series
# DataFrame의 기준인 columns와 Series의 기준인 index가 서로 일치하기 때문에, 의도한 대로 계산 가능
(price_df - price_df.iloc[0]).head()

In [ ]:
# Subtract column Series
# [X] DataFrame의 기준인 columns와 Series의 기준인 index가 서로 불일치 (price_df['open'] - price_df도 마찬가지로 [X] )
(price_df - price_df['Open']).head()

## DataFrame & DataFrame

- index,column 이 일치하는 것 끼리만 element-wise 연산이 이루어지고 나머지는 nan 처리

In [ ]:
price_df - price_df[['Open', 'Low']].iloc[:2] 

## 연산 관련 pandas built-in 함수

- axis란?
- 연산은 기본적으로 "axis를 변형(줄이거나 늘리는)하는 방식" 으로 진행된다.

In [ ]:
#### numpy로 맛보기

import numpy as np

a = np.array([1,2,3])
b = np.array([1,2,3])
a
b

In [ ]:
a + b

In [ ]:
np.sum(
    [a,b],
    axis=0
)

In [ ]:
np.sum(
    [a,b],
    axis=1
)

In [ ]:
[      0  1  2
   0  [1, 2, 2],
   1  [1, 2, 2]
]

In [ ]:
 axis=0 --->       0            1
 axis=1 --->    0  1  2      0  1  2
            [  [1, 2, 2],   [1, 2, 2] ]

In [ ]:
 axis=0 --->              0                        1
 axis=1 --->      0       1     2          0       1      2
 axis=2 --->     0  1    0, 1    0        0, 1    0, 1    0, 1
            [  [[1, 2], [2, 4], [2]],   [[1, 2], [2, 2], [2, 2]] ]

- Note: shift+tab을 이용해서 주석 설명을 보는 것이 가장 정확함



In [ ]:
df.head(2)

In [ ]:
df[['순이익률(%)', 'PER(배)']].sum()  # default axis=0

In [ ]:
df[['순이익률(%)', 'PER(배)']].mean()  # default axis=0

In [ ]:
# DataFrame - Series의 형태
# df's columns: o,h,l,c
# series's index: o,h,l,c
# => mean()값을 통해 Normalizing
(price_df - price_df.mean()).head()

- 아래 구문은 연산 불가능했었음

In [ ]:
close_series = price_df['Close']
price_df - close_series

- -> 하지만 DataFrame이 제공하는 연산관련 함수를 이용하면 가능!

In [ ]:
price_df.head()

In [ ]:
price_df.sub(close_series, axis=0).head()

- `sub()`의 경우 descrption에  'For Series input, axis to match Series index on'라고 써있음
    - axis=0 or 1은 무조건 descrption (shift + tab) 먼저 보고 판단하고 그 후에 "axis는 해당 axis를 변형(줄이거나, 늘리는 것)" 적용하기

- “A simple brute force solution of trying both directions until achieving the desired result is one possibility”

In [ ]:
price_df[['Open', 'Close']].sum(axis=1).head()
price_df[['Open', 'Close']].sum(axis=0).head()

## Example

In [ ]:
b = pd.read_csv("my_data/multi_price.csv", index_col=[0])

In [ ]:
b.head()

In [ ]:
b.loc["2018-08-09"]

In [ ]:
momentum_series = b.loc["2018-08-09"] / b.loc["2017-08-09"] - 1

In [ ]:
momentum_series.nlargest(3)

# Handling nan value

In [ ]:
None == None

In [ ]:
np.nan == np.nan

In [ ]:
5 < np.nan

In [ ]:
5 >= np.nan

In [ ]:
5 == np.nan

- 아래 operation만 True를 return함 

In [ ]:
np.nan != 5

In [ ]:
df1 = pd.DataFrame(
    {
        'a':[1,2,3],
        'b':[np.nan, 4, np.nan],
    }
)
df1

## nan이 있을 때의 boolean series 

In [ ]:
df1['b'] == df1['b']

In [ ]:
df1.ge(2)   # Same with (df1 >= 2)
df1.le(2)

- 따라서 아래와같은 구문은 위험할 수 있음

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['PER(배)'].count()

In [ ]:
df['PER(배)'] > 1

- 주의: 아래처럼 checking하면 `np.nan`의 갯수가 0개로 나옴

In [ ]:
(df['PER(배)'] == np.nan).any()

## Nan checking

### For Series


In [ ]:
df['순이익률(%)'].hasnans

In [ ]:
# Generate boolean series
df['순이익률(%)'].isnull()

In [ ]:
df['순이익률(%)'].isna().any()

In [ ]:
df['순이익률(%)'].isna().sum()

### For DataFrame

In [ ]:
df.isnull().head()

In [ ]:
a = df.isnull()

In [ ]:
a.any(axis=0)

In [ ]:
df.isnull().any().any()   

In [ ]:
df.isnull().any().all()   

- 참고: 두 개의 DataFrame이 같은지 판단하려면, `equals`를 사용하자

In [ ]:
df1['b'] == df1['b']

In [ ]:
df1['b'].equals(df1['b'])

### Example

In [ ]:
_df = pd.DataFrame({'a':[1,np.nan,3], 'b':[np.nan, 2, 3]})
_df.head()

- 둘다 nan이 아닌 값들만 추출

In [ ]:
_df['a'].notnull()
_df['b'].notnull()

In [ ]:
# 1.
_df[ _df['a'].notnull() & _df['b'].notnull() ]

In [ ]:
# 2.

In [ ]:
_df.notnull().all(axis=1)

In [ ]:
_df[_df.notnull().all(axis=1)]

In [ ]:
# 3.
_df.dropna()

In [ ]:
# subset에 있는 컬럼 중에 하나라도(혹은 전부, arg로 선택가능) null이면 drop한다
_df.dropna(subset=['a'])